In [1]:
import pandas as pd

# Load your CSV file
df = pd.read_csv('Karnataka_Datasets/Across/Karnataka_Dataset_Across_Regions.csv')  # Replace with your file path

# Print unique values in the 'Crop_Name' column
unique_crops = df['Crop_Name'].dropna().unique()
print("Unique Crop Names:")
for crop in unique_crops:
    print(crop)


Unique Crop Names:
Maize
Rose
Redgram
Eucalyptus
Ragi
Coconut
Banana
Potato
Avare
Jowar
Sapota
Grapes
Bajra
Guava
Onion
Mangoes
Lemon
Paddy
Arecanut
Black Pepper
Sugarcane


# SAVING ONLY KHARIF SEASON DATA

In [2]:
import pandas as pd
import re

# Load the original CSV
df = pd.read_csv("Karnataka_Datasets/Across/S2/Merged_Karnataka_S2.csv")

# Function to check if a column name ends with _<number> and is in the range 1 to 37
def keep_column(col):
    match = re.match(r'.*_(\d+)$', col)
    if match:
        num = int(match.group(1))
        return num <= 37
    return True  # Keep non-matching columns (like Latitude, Crop_Name, etc.)

# Filter columns
filtered_cols = [col for col in df.columns if keep_column(col)]
df_filtered = df[filtered_cols]

# Save to new CSV
df_filtered.to_csv("Karnataka_Datasets/Across/S2/Merged_Karnataka_S2_Kharif.csv", index=False)


In [4]:
import pandas as pd

# Load the CSV file (replace with your actual filename)
df = pd.read_csv('Karnataka_Datasets/Across/S2/clustered_crops_filtered/Karnataka_Kharif_S2_Data.csv')  # e.g., 'crops.csv'

# Define structure categories and corresponding crops
structure_map = {
    'Tree': ['Guava', 'Mangoes', 'Sapota', 'Eucalyptus', 'Arecanut', 'Coconut', 'Lemon', 'Banana'],
    'Shrub': ['Rose', 'Redgram', 'Avare'],
    'Grass Crop': ['Maize', 'Jowar', 'Bajra', 'Ragi', 'Paddy', 'Sugarcane'],
    'Root/Bulb Crop': ['Potato', 'Onion'],
    'Climber/Vine': ['Grapes', 'Black Pepper']
}

# Assign numeric labels to each structure type
structure_to_numeric = {
    'Tree': 1,
    'Shrub': 2,
    'Grass Crop': 3,
    'Root/Bulb Crop': 4,
    'Climber/Vine': 5
}

# Create crop-to-structure mapping
crop_to_structure = {}
for structure, crops in structure_map.items():
    for crop in crops:
        crop_to_structure[crop] = structure

# Map structure and numeric structure to the DataFrame
df['Structure'] = df['Crop_Name'].map(crop_to_structure)
df['Structure_Numeric'] = df['Structure'].map(structure_to_numeric)

# Print result or save to CSV
print(df[['Crop_Name', 'Structure', 'Structure_Numeric']])

# Optional: save the result
df.to_csv('Karnataka_Datasets/Across/S2/Merged_Karnataka_S2_Kharif_Mapped.csv', index=False)


       Crop_Name   Structure  Structure_Numeric
0       Arecanut        Tree                  1
1       Arecanut        Tree                  1
2       Arecanut        Tree                  1
3       Arecanut        Tree                  1
4       Arecanut        Tree                  1
...          ...         ...                ...
84285  Sugarcane  Grass Crop                  3
84286  Sugarcane  Grass Crop                  3
84287  Sugarcane  Grass Crop                  3
84288  Sugarcane  Grass Crop                  3
84289  Sugarcane  Grass Crop                  3

[84290 rows x 3 columns]
